<a href="https://colab.research.google.com/github/schavan023/Superbugs_winning/blob/main/Superbugs_winning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load Excel file
df_dd = pd.read_excel('/content/DataDictionary.xlsx')

# Preview
df_dd.head()

# Load the Excel file
df_id = pd.read_excel('/content/IsolateData.xlsx')

# Preview the first few rows
df_id.head()



FileNotFoundError: [Errno 2] No such file or directory: '/content/DataDictionary.xlsx'

In [ ]:
df_id.info()
df_id.shape
df_id.columns


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8261 entries, 0 to 8260
Columns: 148 entries, Specimen ID to TIO ConclPred
dtypes: float64(38), int64(3), object(107)
memory usage: 9.3+ MB


Index(['Specimen ID', 'NCBI Accession Number', 'WGS ID', 'AST Approved',
       'WGS Approved', 'Genus', 'Species', 'Serotype', 'Data Year',
       'Region Name',
       ...
       'TEL Concl', 'TEL ConclPred', 'TET Equiv', 'TET Rslt', 'TET Concl',
       'TET ConclPred', 'TIO Equiv', 'TIO Rslt', 'TIO Concl', 'TIO ConclPred'],
      dtype='object', length=148)

In [ ]:
df_id_clean = df_id.copy()
df_id_clean.columns = df_id_clean.columns.str.strip().str.lower().str.replace(' ', '_')


In [ ]:
# 1. Filter for AST-approved Salmonella isolates with AMP results
df_amp = df_id_clean[
    (df_id_clean['ast_approved'].str.lower() == 'yes') &
    (df_id_clean['genus'].str.lower() == 'salmonella') &
    (df_id_clean['amp_concl'].notna())
]

# 2. Create a binary outcome column
df_amp['amp_resistant'] = df_amp['amp_concl'].apply(lambda x: 1 if x == 'R' else 0)

# 3. Preview your new target variable
df_amp['amp_resistant'].value_counts()


,count
amp_resistant,
0,7134
1,1127


In [ ]:
# 1. Select features and target
features = [
    'data_year',
    'region_name',
    'serotype',
    'specimen_source'
]

target = 'amp_resistant'

# 2. Keep only the columns we need
df_model = df_amp[features + [target]].copy()

# 3. Drop rows with any missing values in these columns
df_model.dropna(inplace=True)

# 4. Confirm the shape of the clean dataset
print("Clean dataset shape:", df_model.shape)
df_model.head()


Clean dataset shape: (8239, 5)


,data_year,region_name,serotype,specimen_source,amp_resistant
0,2008,Region 6,Typhi,Blood,1
1,2009,Region 9,Typhi,Stool,0
2,2009,Region 2,Typhi,Blood,1
3,2000,Region 2,Typhi,Other,0
4,2010,Region 10,Typhi,Blood,0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# 1. Separate features (X) and target (y)
X = df_model.drop(columns=['amp_resistant'])
y = df_model['amp_resistant']

# 2. One-hot encode categorical variables
X_encoded = pd.get_dummies(X, drop_first=True)

# 3. Confirm encoding worked
print("Encoded shape:", X_encoded.shape)
X_encoded.head()


Encoded shape: (8239, 42)


,data_year_2000,data_year_2001,data_year_2002,data_year_2003,data_year_2004,data_year_2005,data_year_2006,data_year_2007,data_year_2008,data_year_2009,...,region_name_Region 8,region_name_Region 9,specimen_source_Blood,specimen_source_Gall Bladder,specimen_source_Not Given,specimen_source_Other,specimen_source_Stool,specimen_source_Unknown,specimen_source_Urine,specimen_source_Wound
0,False,False,False,False,False,False,False,False,True,False,...,False,False,True,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,True,...,False,True,False,False,False,False,True,False,False,False
2,False,False,False,False,False,False,False,False,False,True,...,False,False,True,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
